# Combining `CurveFits` objects

We can also combine together `CurveFits` objects for distinct serum/viruses/replicates using the `CurveFits.combineCurveFits` method.
This is useful if you are fitting large datasets in chunks, and then want to combine all the results.

Here is an example/test:

In [1]:
import pandas as pd

import neutcurve

Read in the data:

In [2]:
fi6v3_datafile = "Doud_et_al_2018-neutdata.csv"
data = pd.read_csv(fi6v3_datafile)

Create a `CurveFits` object with all the data:

In [3]:
fits = neutcurve.CurveFits(data)
_ = fits.fitParams(average_only=False)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Now split the data to make different `CurveFits` which we can combine:

In [4]:
# split the data in half
data1 = data[(data["replicate"] == 1) | (data["serum"] == "H17-L19")]
data2 = data[(data["replicate"] != 1) & (data["serum"] != "H17-L19")]

# make a second split that is **not** disjoint from the first one
data2_invalid = data[data["replicate"] != 1]

# make fits to each of these
fit1 = neutcurve.CurveFits(data1)
_ = fit1.fitParams(average_only=False)
fit2 = neutcurve.CurveFits(data2)
_ = fit2.fitParams(average_only=False)
fit2_invalid = neutcurve.CurveFits(data2_invalid)
_ = fit2_invalid.fitParams(average_only=False)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)
/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Combine two fits using `CurveFits.combineCurveFits` that should yield an object the same as `fits`:

In [5]:
combined_fits = neutcurve.CurveFits.combineCurveFits([fit1, fit2])

pd.testing.assert_frame_equal(
    fits.fitParams(average_only=False),
    combined_fits.fitParams(average_only=False),
)

/home/jbloom/neutcurve/neutcurve/hillcurve.py:738: RuntimeWarning: invalid value encountered in power
  return b + (t - b) / (1 + (c / m) ** s)


Combine fits only for certain sera:

In [6]:
(
    neutcurve.CurveFits.combineCurveFits(
        [fit1, fit2],
        sera=["FI6v3"],
        viruses=["WT", "P80D", "V135T"],
        serum_virus_replicates_to_drop=[
            ("FI6v3", "P80D", "2"),
            ("FI6v3", "V135T", "1"),
            ("FI6v3", "V135T", "2"),
            ("FI6v3", "V135T", "3"),
        ],
    ).fitParams(average_only=False)
)

,serum,virus,replicate,nreplicates,ic50,ic50_bound,ic50_str,midpoint,slope,top,bottom
0,FI6v3,WT,1,<NA>,0.016702,interpolated,0.0167,0.016702,2.504670,1,0
1,FI6v3,WT,2,<NA>,0.019020,interpolated,0.019,0.019020,2.512733,1,0
2,FI6v3,WT,3,<NA>,0.015167,interpolated,0.0152,0.015167,1.877995,1,0
3,FI6v3,WT,average,3,0.017029,interpolated,0.017,0.017029,2.279316,1,0
4,FI6v3,P80D,1,<NA>,0.012115,interpolated,0.0121,0.012115,2.024698,1,0
5,FI6v3,P80D,3,<NA>,0.012835,interpolated,0.0128,0.012835,2.059061,1,0
6,FI6v3,P80D,average,2,0.012472,interpolated,0.0125,0.012472,2.035435,1,0


Make sure we cannot combine two fits with overlapping entries, this should give an error due to shared serum/virus/replicates:

In [7]:
# NBVAL_RAISES_EXCEPTION

neutcurve.CurveFits.combineCurveFits([fit1, fit2_invalid])

ValueError: duplicated sera/virus/replicate in `curvefits_list`